In [38]:
pip install nltk rouge-score bert-score

In [39]:
ref_summary="""
financial literacy is the ability to understand and make use of a variety of financial skills, including personal financial management, budgeting, and investing.
educating yourself on these topics also involves learning how money works, setting and achieving financial goals, becoming aware of unethical/discriminatory financial practices, and managing financial challenges that life throws your way.
personal finance is about making and meeting your financial goals, whether you want to own a home, help other members of your family, save for your children’s college education, support causes that you care about, plan for retirement, or anything else.
"""

In [40]:
from rouge_score import rouge_scorer
from bert_score import score as bert_score

def calculate_summary_scores(reference_text, generated_summary):
    # Prepare inputs
    reference_list = [reference_text.split()]
    generated_list = generated_summary.split()

    # ROUGE Scores
    rouge_scorer_instance = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    rouge_scores = rouge_scorer_instance.score(reference_text, generated_summary)

    # BERTScore
    P, R, F1 = bert_score([generated_summary], [reference_text], lang="en")

    # Display results
    print("Summary Evaluation Metrics:")
    print("-" * 30)
    print("\nROUGE Scores:")
    print(f"  ROUGE-1: Precision: {rouge_scores['rouge1'].precision:.4f}, Recall: {rouge_scores['rouge1'].recall:.4f}, F1: {rouge_scores['rouge1'].fmeasure:.4f}")
    print(f"  ROUGE-2: Precision: {rouge_scores['rouge2'].precision:.4f}, Recall: {rouge_scores['rouge2'].recall:.4f}, F1: {rouge_scores['rouge2'].fmeasure:.4f}")
    print(f"  ROUGE-L: Precision: {rouge_scores['rougeL'].precision:.4f}, Recall: {rouge_scores['rougeL'].recall:.4f}, F1: {rouge_scores['rougeL'].fmeasure:.4f}")
    print("\nBERT Score:")
    print(f"  Precision: {P.mean().item():.4f}")
    print(f"  Recall: {R.mean().item():.4f}")
    print(f"  F1 Score: {F1.mean().item():.4f}")

## References:

https://huggingface.co/spaces/evaluate-metric/bertscore

In [41]:
text = """Financial literacy is the ability to understand and make use of a variety of financial skills, including personal financial management, budgeting, and investing. It also means comprehending certain financial principles and concepts, such as the time value of money, compound interest, managing debt, and financial planning.

Achieving financial literacy can help individuals to avoid making poor financial decisions. It can help them become self-sufficient and achieve financial stability. Key steps to attaining financial literacy include learning how to create a budget, track spending, pay off debt, and plan for retirement.

Educating yourself on these topics also involves learning how money works, setting and achieving financial goals, becoming aware of unethical/discriminatory financial practices, and managing financial challenges that life throws your way.
Personal Finance Basics
Personal finance is where financial literacy translates into individual financial decision-making. How do you manage your money? Which savings and investment vehicles are you using? Personal finance is about making and meeting your financial goals, whether you want to own a home, help other members of your family, save for your children’s college education, support causes that you care about, plan for retirement, or anything else.

Among other topics, it encompasses banking, budgeting, handling debt and credit, and investing. Let’s take a look at these basics to get you started.

Introduction to Bank Accounts
A bank account is typically the first financial account that you’ll open. Bank accounts can hold and build the money you'll need for major purchases and life events. Here’s some background on bank accounts and why they are step one in creating a stable financial future.

"""

In [42]:
import re
# Remove leading/trailing whitespace and extra newline characters
text = text.strip()
# Remove commented lines
text = re.sub(r'^\s*#.*$', '', text, flags=re.MULTILINE)

# Remove extra whitespace within the text
text = re.sub(r'\s+', ' ', text)
# Convert to lowercase
text = text.lower()

In [43]:
print(text)

financial literacy is the ability to understand and make use of a variety of financial skills, including personal financial management, budgeting, and investing. it also means comprehending certain financial principles and concepts, such as the time value of money, compound interest, managing debt, and financial planning. achieving financial literacy can help individuals to avoid making poor financial decisions. it can help them become self-sufficient and achieve financial stability. key steps to attaining financial literacy include learning how to create a budget, track spending, pay off debt, and plan for retirement. educating yourself on these topics also involves learning how money works, setting and achieving financial goals, becoming aware of unethical/discriminatory financial practices, and managing financial challenges that life throws your way. personal finance basics personal finance is where financial literacy translates into individual financial decision-making. how do you 

**Google T5**

In [44]:
import torch

from transformers import AutoTokenizer, AutoModelWithLMHead

In [45]:
tokenizer = AutoTokenizer.from_pretrained('t5-base')
model = AutoModelWithLMHead.from_pretrained('t5-base', return_dict=True)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1833: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


**Prompt 1**

In [46]:
inputs = tokenizer.encode("Summarize the text and give key points briefly:   " + text,
return_tensors='pt',
max_length=512,
truncation=True)

In [47]:
summary_ids = model.generate(inputs, max_length=60, min_length=40)
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

In [48]:
summary

'financial literacy is the ability to understand and make use of a variety of financial skills . personal finance is about making and meeting your financial goals . bank accounts are typically the first financial account that you’ll open .'

In [49]:
print("Scores for T5 Prompt 1")
calculate_summary_scores(text, summary)

Scores for T5 Prompt 1


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Summary Evaluation Metrics:
------------------------------

ROUGE Scores:
  ROUGE-1: Precision: 1.0000, Recall: 0.1397, F1: 0.2452
  ROUGE-2: Precision: 0.8919, Recall: 0.1218, F1: 0.2143
  ROUGE-L: Precision: 0.9737, Recall: 0.1360, F1: 0.2387

BERT Score:
  Precision: 0.9233
  Recall: 0.8288
  F1 Score: 0.8735


**Prompt 2**

In [50]:
inputs = tokenizer.encode("Summarize and give as key points: " + text,
return_tensors='pt',
max_length=512,
truncation=True)

In [51]:
summary_ids = model.generate(inputs, max_length=400, min_length=300)
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

In [52]:
summary

'financial literacy is the ability to understand and make use of a variety of financial skills . it includes personal financial management, budgeting, and investing . personal finance is about making and meeting your financial goals . personal finance includes banking, budgeting, handling debt and credit, and investing . bank accounts are the first financial account that you’ll open . bank accounts are a great way to save money and build your financial future .. bank accounts are a great way to save money and build your financial future .. bank accounts                                                                                                                                         '

In [53]:
print("Scores for T5 Prompt 2")
calculate_summary_scores(text, summary)

Scores for T5 Prompt 2


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Summary Evaluation Metrics:
------------------------------

ROUGE Scores:
  ROUGE-1: Precision: 0.9302, Recall: 0.2941, F1: 0.4469
  ROUGE-2: Precision: 0.6118, Recall: 0.1919, F1: 0.2921
  ROUGE-L: Precision: 0.7093, Recall: 0.2243, F1: 0.3408

BERT Score:
  Precision: 0.9082
  Recall: 0.8522
  F1 Score: 0.8793


# Hugging Face

In [54]:
from transformers import pipeline

**DistilBERT**

In [55]:

summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6")
# Generate the summary
summary = summarizer(text, max_length=100, min_length=90, do_sample=False)
# Print the summary
summary=summary[0]['summary_text']

In [56]:
summary

' financial literacy is the ability to understand and make use of a variety of financial skills, including personal financial management, budgeting, and investing . it also means comprehending certain financial principles and concepts, such as the time value of money, compound interest, managing debt, and financial planning . educating yourself on these topics also involves learning how money works, setting and achieving financial goals, becoming aware of unethical financial practices, and managing financial challenges that life throws your way .'

In [57]:
print("Scores for DistilBERT")
calculate_summary_scores(text, summary)

Scores for DistilBERT


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Summary Evaluation Metrics:
------------------------------

ROUGE Scores:
  ROUGE-1: Precision: 1.0000, Recall: 0.2831, F1: 0.4413
  ROUGE-2: Precision: 0.9737, Recall: 0.2731, F1: 0.4265
  ROUGE-L: Precision: 1.0000, Recall: 0.2831, F1: 0.4413

BERT Score:
  Precision: 0.9682
  Recall: 0.8692
  F1 Score: 0.9160


**T5 Small**

In [58]:
summarizer = pipeline("summarization", model="t5-small")
# Generate the summary
summary = summarizer(text, max_length=100, min_length=90, do_sample=False)
# Print the summary
summary=summary[0]['summary_text']

In [59]:
summary

'financial literacy is the ability to understand and make use of a variety of financial skills, including personal financial management, budgeting, and investing . key steps to attaining financial literacy include learning how to create a budget, track spending, pay off debt, and plan for retirement . educating yourself on these topics also involves learning how money works, setting and achieving financial goals, and managing financial challenges that life throws your way .'

In [60]:
print("Scores for T5 Small")
calculate_summary_scores(text, summary)

Scores for T5 Small


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Summary Evaluation Metrics:
------------------------------

ROUGE Scores:
  ROUGE-1: Precision: 1.0000, Recall: 0.2574, F1: 0.4094
  ROUGE-2: Precision: 0.9710, Recall: 0.2472, F1: 0.3941
  ROUGE-L: Precision: 1.0000, Recall: 0.2574, F1: 0.4094

BERT Score:
  Precision: 0.9601
  Recall: 0.8662
  F1 Score: 0.9107


**PEGASUS**

In [61]:
summarizer = pipeline("summarization", model="google/pegasus-cnn_dailymail")
# Generate the summary
summary = summarizer(text, max_length=100, min_length=90, do_sample=False)
# Print the summary
summary=summary[0]['summary_text']

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [62]:
summary

'financial literacy is the ability to understand and make use of a variety of financial skills .<n>key steps to attaining financial literacy include learning how to create a budget, track spending, pay off debt, and plan for retirement .<n>personal finance is about making and meeting your financial goals, whether you want to own a home, help other members of your family, save for your children’s college education, plan for retirement, or anything else .'

In [63]:
print("Scores for Pegasus")
calculate_summary_scores(text, summary)

Scores for Pegasus


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Summary Evaluation Metrics:
------------------------------

ROUGE Scores:
  ROUGE-1: Precision: 0.9737, Recall: 0.2721, F1: 0.4253
  ROUGE-2: Precision: 0.9333, Recall: 0.2583, F1: 0.4046
  ROUGE-L: Precision: 0.9737, Recall: 0.2721, F1: 0.4253

BERT Score:
  Precision: 0.9367
  Recall: 0.8651
  F1 Score: 0.8995


 **LED (Longformer Encoder-Decoder)**

In [64]:
summarizer = pipeline("summarization", model="allenai/led-base-16384")
# Generate the summary
summary = summarizer(text, max_length=100, min_length=90, do_sample=False)
# Print the summary
summary=summary[0]['summary_text']

In [65]:
summary

'financial literacy is the ability to understand and make use of a variety of financial skills, including personal financial management, budgeting, and investing. it also means comprehending certain financial principles and concepts, such as the time value of money, compound interest, managing debt, and financial planning. achieving financial literacy can help individuals to avoid making poor financial decisions. it can help them become self-sufficient and achieve financial stability. key steps to attaining financial literacy include learning how to create a budget,'

In [66]:
print("Scores for LED")
calculate_summary_scores(text, summary)

Scores for LED


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Summary Evaluation Metrics:
------------------------------

ROUGE Scores:
  ROUGE-1: Precision: 1.0000, Recall: 0.3015, F1: 0.4633
  ROUGE-2: Precision: 1.0000, Recall: 0.2989, F1: 0.4602
  ROUGE-L: Precision: 1.0000, Recall: 0.3015, F1: 0.4633

BERT Score:
  Precision: 0.9845
  Recall: 0.8743
  F1 Score: 0.9261


**BART**

In [67]:
text

"financial literacy is the ability to understand and make use of a variety of financial skills, including personal financial management, budgeting, and investing. it also means comprehending certain financial principles and concepts, such as the time value of money, compound interest, managing debt, and financial planning. achieving financial literacy can help individuals to avoid making poor financial decisions. it can help them become self-sufficient and achieve financial stability. key steps to attaining financial literacy include learning how to create a budget, track spending, pay off debt, and plan for retirement. educating yourself on these topics also involves learning how money works, setting and achieving financial goals, becoming aware of unethical/discriminatory financial practices, and managing financial challenges that life throws your way. personal finance basics personal finance is where financial literacy translates into individual financial decision-making. how do you

In [68]:
len(text.split(" "))

264

In [75]:
summarizer = pipeline("summarization", model="facebook/bart-large-xsum")
# Generate the summary
summary = summarizer(text, max_length=100, min_length=90, do_sample=False)
# Print the summary
summary=summary[0]['summary_text']

In [76]:
summary

'If you’re looking for a way to improve your financial literacy, here’s what you need to know about personal finance. and how you can get started. in our series of articles, we’ll be looking at how to get started with personal finance, and how to help you achieve financial stability and self-sufficiency, as well as how to set and achieve goals, and what you can do to help yourself and your family.'

In [78]:
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
# Generate the summary
summary = summarizer(text, max_length=100, min_length=90, do_sample=False)
# Print the summary
summary=summary[0]['summary_text']

In [79]:
summary

' financial literacy is the ability to understand and make use of a variety of financial skills, including personal financial management, budgeting, and investing. achieving financial literacy can help individuals to avoid making poor financial decisions. It can help them become self-sufficient and achieve financial stability. educating yourself on these topics also involves learning how money works, setting and achieving financial goals, becoming aware of unethical/discriminatory financial practices, and managing financial challenges that life throws your way.'

**PEGASUS**

In [80]:
summarizer = pipeline("summarization", model="google/pegasus-cnn_dailymail")
# Generate the summary
summary = summarizer(text, max_length=100, min_length=90, do_sample=False)
# Print the summary
summary=summary[0]['summary_text']

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [82]:
summary

'financial literacy is the ability to understand and make use of a variety of financial skills .<n>key steps to attaining financial literacy include learning how to create a budget, track spending, pay off debt, and plan for retirement .<n>personal finance is about making and meeting your financial goals, whether you want to own a home, help other members of your family, save for your children’s college education, plan for retirement, or anything else .'